In [5]:
import os

In [6]:
%pwd

'c:\\Users\\Admin\\Projects\\ML Projects\\stock_sentiment\\src'

In [3]:
os.chdir('../src')

In [7]:
%pwd

'c:\\Users\\Admin\\Projects\\ML Projects\\stock_sentiment\\src'

In [8]:
from config_entity import DatabaseConfig